# Use Case Notebook by Lars Werner (3066030)

### This Use Case Notebook is an example, how you can load Recordings without Ground Truth in EDF Format into Spikeinterface with MNE and for comparing the results with metrics. You will need a EDF Recording with an Probe File for the locations of the electrodes. If your Recording has other formats, you will need to look up if SpikeInterface or MNE does support your format. If not, you will need to find a different library, that supports your format.

In [ ]:
import mne
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
import SpikeSortingPipeline as ssp
import numpy as np
%matplotlib notebook

### Loading EDF File and convert it into numpy array

In [ ]:
file_path = "data/NP_2010.02.12_10.edf"

edf_raw = mne.io.read_raw_edf(file_path, preload=True)
hz = 1 / (edf_raw.times[1] - edf_raw.times[0])
edf_data, times = edf_raw[:, :]

print(type(edf_raw))
print(hz)
print(type(edf_data))
print(edf_data)
print(np.shape(edf_data))

### Loading numpy Array into SpikeInterface with NumpyRecordingExtractor, loading the probe file, adding the recording to an RecordingList and printing some data.

In [ ]:
recording = se.NumpyRecordingExtractor(timeseries=edf_data, sampling_frequency=hz)
recording = recording.load_probe_file("data/probe.prb")
recordings = ssp.createRecordingList()
recordings = ssp.addRecording(recordings, recording, "", "Own Data")
print(recordings, "\n")
ssp.printRecordingData(recordings)

### This is for printing the traces of the recording and to check if the recording has Nan values

In [ ]:
data = recordings[0][1].get_traces()
print(recording.get_shared_channel_property_names())
print(type(data))
print(data)

array_sum = np.sum(data)
print("Has Nan Value: ", np.isnan(array_sum))


### We dont have any Ground Truth Data, so we can only use the Print Functions, that only uses the RecordingExtractor.


In [ ]:
ssp.printTimeseries(recordings)
ssp.printElectrodeGeometry(recordings)

### You can try running my RunSpikeSorting Method, but if it fails you will need to exlucde some installed sorters. Therefore you can use the code below. You can exlude Sorters by slicing the sorter_list.

In [ ]:
spike_sorting = ss.run_sorters(sorter_list = ss.installed_sorters()[1:2] + ss.installed_sorters()[3:], 
                                recording_dict_or_list = [recordings[0][1]], 
                                working_folder='working_folder_use_case',
                                mode='overwrite'
                               )

### Loads your working folder where your sorting results are saved. You dont need to run SpikeSorting every time.

In [ ]:
spike_sorting = ss.collect_sorting_outputs("working_folder_use_case")

### Creating and printing the SorterList

In [ ]:
sorter_list = ssp.createSorterList(recordings,spike_sorting)
ssp.printSorterList(sorter_list)

### Because we don't have any Ground Truth Data, we can't use the CompareWithGroundTruth or PrintPerformance Functions. So we need to use quality metrics or the compareSorters functions to evaluate the performance of the recordings. SpikeInterface can compute some quality metrics and the code below shows you, how you can let them get computed.

In [ ]:
for sorter in sorter_list[0][1]:
    
    metrics = st.validation.compute_quality_metrics(sorting=sorter[1], recording=sorter_list[0][0][1],
                                                metric_names=['num_spikes', 
                                                              'firing_rate',
                                                              'presence_ratio',
                                                              'isi_violation',
                                                              'amplitude_cutoff',
                                                              'snr',
                                                              'max_drift', 
                                                              'cumulative_drift', 
                                                              'silhouette_score',
                                                              #'isolation_distance',
                                                              #'l_ratio', 
                                                              #'d_prime',
                                                              #'noise_overlap',
                                                              #'nn_hit_rate', 
                                                              #'nn_miss_rate'
                                                                ], 
                                                              as_dataframe=True)
    print("Metrics from:", sorter[0])
    print(type(metrics))
    print(metrics)

### This is for comparing the sorter results with each other

In [ ]:
ssp.compareSorters(sorter_list[0])